In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    f1_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef,
    classification_report, confusion_matrix, log_loss
)
import plotly.graph_objects as go
from cassandra.cluster import Cluster
from datetime import datetime

In [5]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('data_stock')

In [6]:

rows_price = session.execute("""
    SELECT * FROM candlestick_data ALLOW FILTERING
""")
df_candlestick_latest = pd.DataFrame(rows_price)
df_candlestick_latest['time'] = pd.to_datetime(df_candlestick_latest['time'])
df_candlestick_latest = df_candlestick_latest.sort_values(['symbol', 'time'])

In [7]:
import ta
def compute_technical_grades(df_candle, band_pct=0.015):
    df_result = []

    for symbol, df_sym in df_candle.groupby('symbol'):
        df_sym = df_sym.sort_values('time').copy()
        df_sym['ema5'] = df_sym['close_price'].ewm(span=5, adjust=False).mean()
        df_sym['ema15'] = df_sym['close_price'].ewm(span=15, adjust=False).mean()
        df_sym['ema35'] = df_sym['close_price'].ewm(span=35, adjust=False).mean()
        df_sym['ema89'] = df_sym['close_price'].ewm(span=89, adjust=False).mean()
        df_sym['ema200'] = df_sym['close_price'].ewm(span=200, adjust=False).mean()
        df_sym['rsi'] = ta.momentum.RSIIndicator(close=df_sym['close_price'], window=14).rsi()

        # ใช้แท่งล่าสุดในการประเมิน
        row = df_sym.iloc[-1]
        try:
            c = row['close_price']
            r = row['rsi']
            e5, e15, e35, e89, e200 = row['ema5'], row['ema15'], row['ema35'], row['ema89'], row['ema200']

            if c >= e5 and r >= 70:
                grade = 'a'
            elif c >= e35 and e35 >= e89:
                grade = 'b'
            elif c >= e89 and (max([e5,e15,e35,e89]) - min([e5,e15,e35,e89])) / np.mean([e5,e15,e35,e89]) <= band_pct:
                grade = 'c'
            elif c < e89 and c < e200 and e89 < e200:
                grade = 'd'
            elif c < e5 < e15 < e35 < e89 < e200 and r <= 30:
                grade = 'e'
            else:
                grade = None
        except:
            grade = None

        df_result.append({'symbol': symbol, 'trend_grade': grade})

    return pd.DataFrame(df_result)

In [8]:
# 🚨 สมมุติว่าคุณมี DataFrame แบบนี้
# df_candlestick_latest.columns = ['symbol', 'time', 'close_price', 'open', 'high', 'low', 'volume']

# 🧠 เรียกใช้ฟังก์ชันของคุณ
df_graded = compute_technical_grades(df_candlestick_latest)

# ✅ ดูผลลัพธ์
df_graded.head(10)  # แสดง DataFrame ที่มีคอลัมน์ 'symbol' และ 'trend_grade'


,symbol,trend_grade
0,24CS,d
1,2S,None
2,3BBIF,b
3,A,d
4,A5,d
5,AAI,d
6,AAV,d
7,ABM,d
8,ACAP,d
9,ACC,d


In [9]:
rows_fin2 = session.execute("""
    SELECT symbol,eps,pe,pbv,percentYield FROM financal_data_fromsettradeAPI ALLOW FILTERING
""")
df_fundamental= pd.DataFrame(rows_fin2)
df_fundamental.head(10)

,symbol,eps,pe,pbv,percentyield
0,PPPM,0.02000,0.00,0.70,0.00
1,TPCH,0.24112,7.90,0.34,13.91
2,KPNREIT,NaN,NaN,0.23,0.00
3,POLY,0.27632,12.89,2.42,6.38
4,QHBREIT,NaN,NaN,0.41,0.00
5,VCOM,0.21428,7.85,1.48,10.79
6,KDH,4.12186,10.62,1.93,3.01
7,NVD,-0.00942,38.87,0.32,0.00
8,JDF,0.04224,12.19,1.35,4.85
9,SVR,-0.03402,0.00,0.34,0.00


In [10]:
rows_price = session.execute("""
    SELECT * FROM candlestick_data ALLOW FILTERING
""")
df_candlestick_latest = pd.DataFrame(rows_price)
df_candlestick_latest['time'] = pd.to_datetime(df_candlestick_latest['time'])
df_candlestick_latest = df_candlestick_latest.sort_values(['symbol', 'time'])

In [11]:
# ✅ 1) ดึงแท่งล่าสุดของแต่ละ symbol
df_latest_candle = (
    df_candlestick_latest.sort_values("time")
    .groupby("symbol", as_index=False)
    .tail(1)  # หรือใช้ .last() ก็ได้หลัง sort แล้ว
)

# ✅ 2) Merge กับข้อมูล fundamental
df = pd.merge(df_fundamental, df_latest_candle, on="symbol", how="inner")

# ✅ 3) คำนวณ marketcap
df["marketcap"] = df["close_price"] * df["volume"]


In [12]:
df.head()

,symbol,eps,pe,pbv,percentyield,time,close_price,high_price,low_price,open_price,value,volume,marketcap
0,PPPM,0.02000,0.00,0.70,0.00,2025-05-28,0.49,0.51,0.48,0.49,0.0,313650,153688.502991
1,TPCH,0.24112,7.90,0.34,13.91,2025-05-28,3.08,3.10,3.04,3.10,0.0,52708,162340.635979
2,KPNREIT,NaN,NaN,0.23,0.00,2025-05-22,2.56,2.56,2.56,2.56,0.0,100,255.999994
3,POLY,0.27632,12.89,2.42,6.38,2025-05-28,6.95,6.95,6.65,6.90,0.0,5100,35444.999027
4,QHBREIT,NaN,NaN,0.41,0.00,2025-05-28,3.44,3.44,3.42,3.44,0.0,140301,482635.448028


In [13]:
# ✅ Clustering กลุ่ม A–E ด้วย KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

features = ['eps', 'pe', 'pbv', 'percentyield', 'marketcap']
X = df[features].dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
kmeans_labels = kmeans.fit_predict(X_scaled)
df.loc[X.index, 'group'] = kmeans_labels

group_map = {i: chr(65+i) for i in range(5)}
df['group'] = df['group'].map(group_map)

In [14]:
df.head(10)

,symbol,eps,pe,pbv,percentyield,time,close_price,high_price,low_price,open_price,value,volume,marketcap,group
0,PPPM,0.02000,0.00,0.70,0.00,2025-05-28,0.49,0.51,0.48,0.49,0.0,313650,153688.502991,A
1,TPCH,0.24112,7.90,0.34,13.91,2025-05-28,3.08,3.10,3.04,3.10,0.0,52708,162340.635979,B
2,KPNREIT,NaN,NaN,0.23,0.00,2025-05-22,2.56,2.56,2.56,2.56,0.0,100,255.999994,NaN
3,POLY,0.27632,12.89,2.42,6.38,2025-05-28,6.95,6.95,6.65,6.90,0.0,5100,35444.999027,B
4,QHBREIT,NaN,NaN,0.41,0.00,2025-05-28,3.44,3.44,3.42,3.44,0.0,140301,482635.448028,NaN
5,VCOM,0.21428,7.85,1.48,10.79,2025-05-28,2.50,2.62,2.50,2.56,0.0,335100,837750.000000,B
6,KDH,4.12186,10.62,1.93,3.01,2025-05-22,85.00,85.00,84.75,85.00,0.0,1200,102000.000000,A
7,NVD,-0.00942,38.87,0.32,0.00,2025-05-28,1.00,1.02,1.00,1.02,0.0,16301,16301.000000,A
8,JDF,0.04224,12.19,1.35,4.85,2025-05-28,1.84,1.85,1.84,1.85,0.0,200,368.000007,B
9,SVR,-0.03402,0.00,0.34,0.00,2025-05-28,0.58,0.59,0.57,0.58,0.0,47400,27491.999209,A


In [15]:
# ✅ 1) สมมติว่า df_graded มี: symbol | trend_grade
#         และ df มี:        symbol | group

# ✅ 2) Merge โดยใช้คอลัมน์ 'symbol'
df_merged = pd.merge(df, df_graded[['symbol', 'trend_grade']], on='symbol', how='inner')

# ✅ 3) สร้างคอลัมน์ quadrant (เช่น Aa, Be, Dc, etc.)
df_merged['quadrant'] = df_merged['group'] + df_merged['trend_grade']


In [16]:
df_merged[['symbol', 'group', 'trend_grade', 'quadrant']].head(10)


,symbol,group,trend_grade,quadrant
0,PPPM,A,None,NaN
1,TPCH,B,d,Bd
2,KPNREIT,NaN,d,NaN
3,POLY,B,d,Bd
4,QHBREIT,NaN,d,NaN
5,VCOM,B,None,NaN
6,KDH,A,d,Ad
7,NVD,A,d,Ad
8,JDF,B,d,Bd
9,SVR,A,d,Ad


In [17]:
df_merged.head(10)

,symbol,eps,pe,pbv,percentyield,time,close_price,high_price,low_price,open_price,value,volume,marketcap,group,trend_grade,quadrant
0,PPPM,0.02000,0.00,0.70,0.00,2025-05-28,0.49,0.51,0.48,0.49,0.0,313650,153688.502991,A,None,NaN
1,TPCH,0.24112,7.90,0.34,13.91,2025-05-28,3.08,3.10,3.04,3.10,0.0,52708,162340.635979,B,d,Bd
2,KPNREIT,NaN,NaN,0.23,0.00,2025-05-22,2.56,2.56,2.56,2.56,0.0,100,255.999994,NaN,d,NaN
3,POLY,0.27632,12.89,2.42,6.38,2025-05-28,6.95,6.95,6.65,6.90,0.0,5100,35444.999027,B,d,Bd
4,QHBREIT,NaN,NaN,0.41,0.00,2025-05-28,3.44,3.44,3.42,3.44,0.0,140301,482635.448028,NaN,d,NaN
5,VCOM,0.21428,7.85,1.48,10.79,2025-05-28,2.50,2.62,2.50,2.56,0.0,335100,837750.000000,B,None,NaN
6,KDH,4.12186,10.62,1.93,3.01,2025-05-22,85.00,85.00,84.75,85.00,0.0,1200,102000.000000,A,d,Ad
7,NVD,-0.00942,38.87,0.32,0.00,2025-05-28,1.00,1.02,1.00,1.02,0.0,16301,16301.000000,A,d,Ad
8,JDF,0.04224,12.19,1.35,4.85,2025-05-28,1.84,1.85,1.84,1.85,0.0,200,368.000007,B,d,Bd
9,SVR,-0.03402,0.00,0.34,0.00,2025-05-28,0.58,0.59,0.57,0.58,0.0,47400,27491.999209,A,d,Ad


In [22]:
import plotly.graph_objects as go
df_result=df_merged
# ✅ จัดกลุ่มหุ้นตาม quadrant
quadrant_groups = df_result.groupby('quadrant')['symbol'].apply(list).to_dict()

# ✅ กำหนดลำดับของแกน
financial_levels = ['A', 'B', 'C', 'D', 'E']  # A = Strong fundamentals
trend_levels = ['a', 'b', 'c', 'd', 'e']      # a = Bullish, e = Crash

# ✅ เตรียมข้อมูลแต่ละเซลล์ในตาราง (5x5)
plot_table = []
for f in financial_levels:
    row = []
    for t in trend_levels:
        code = f + t
        stocks = quadrant_groups.get(code, [])
        if stocks:
            stock_list = "<br>".join(stocks[:10])  # แสดงสูงสุด 10 ตัว
            cell_text = f"<b>{code}</b><br>{stock_list}"
        else:
            cell_text = f"<b>{code}</b><br>-"
        row.append(cell_text)
    plot_table.append(row)

# ✅ สร้างสีของแต่ละแถวตามกลุ่มพื้นฐาน A–E
colors = [[{'A': '#2ecc71', 'B': '#58d68d', 'C': '#f4d03f', 'D': '#f39c12', 'E': '#e74c3c'}[f]]*5 for f in financial_levels]

# ✅ Plotly Table
fig = go.Figure(data=go.Table(
    header=dict(
        values=["Super Bullish", "Up Trend", "Sideway", "Down Trend", "Crash"],
        fill_color="#dcdcdc",
        align="center",
        font=dict(color="black", size=14)
    ),
    cells=dict(
        values=plot_table,
        fill_color=colors,
        align="center",
        height=100,
        font=dict(color="white", size=12)
    )
))

# ✅ ตั้งชื่อกราฟ
fig.update_layout(title="📊 Stock Quadrant Table (Aa–Ee) จาก Rule-Based or Model")
fig.show()
